In [4]:
import numpy as np
import pandas as pd

# Load ratings Change directory where appropriate
directory = ''
ratings = pd.read_csv(directory+'ratings.csv', usecols=['userId', 'movieId', 'rating'])
movies = pd.read_csv(directory+'movies.csv', usecols=['movieId', 'title', 'genres'])
max_userid = ratings['userId'].drop_duplicates().max()+1
max_movieid = ratings['movieId'].drop_duplicates().max() +1
ratings.head()
print(max_userid)
print(max_movieid)

611
193610


In [5]:

from keras.layers import Embedding, Reshape, Concatenate,dot,Input,Dense
from keras.models import Model,clone_model
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint

factors = 100 # The number of dimensional embeddings for movies and users


# Create training set
#shuffled_ratings = ratings.sample(frac=1)

# Shuffling users
#user_ids = shuffled_ratings['userId'].values

# Shuffling movies
#movie_ids = shuffled_ratings['movieId'].values

# Shuffling ratings
#ratings_list = shuffled_ratings['rating'].values


user_ids = ratings['userId'].values
movie_ids = ratings['movieId'].values
ratings_list = ratings['rating'].values


In [6]:
# user_layers is the embedding layer that creates an User by latent factors matrix.

user_input = Input(shape=(1,), name = 'user_layers')
user_middle  = Embedding(max_userid, factors,input_length=1)(user_input)
user_layers  = Reshape(target_shape=(factors,))(user_middle)


# movie_layers is the embedding layer that creates a Movie by latent factors matrix.
movie_input = Input(shape=(1,), name = 'movie_layers')
movie_middle  = Embedding(max_movieid, factors,input_length=1)(movie_input)
movie_layers  = Reshape(target_shape=(factors,))(movie_middle)

output = dot([user_layers,movie_layers], axes=1)

model = Model(inputs=[user_input,movie_input], 
                  outputs=output)
training_model = clone_model(model)
#model = Model(inputs=[user_layers, movie_layers],output=dot_product)
#model.Dot(left, right)
model.compile(loss='mse', optimizer='adamax')
# Callbacks monitor the validation loss
# Save the model weights each time the validation loss has improved


W0801 12:50:44.786769 14976 deprecation_wrapper.py:119] From C:\Users\HP\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0801 12:50:45.114818 14976 deprecation_wrapper.py:119] From C:\Users\HP\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0801 12:50:45.114818 14976 deprecation_wrapper.py:119] From C:\Users\HP\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0801 12:50:45.208555 14976 deprecation_wrapper.py:119] From C:\Users\HP\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.tr

In [ ]:
callbacks = [EarlyStopping('val_loss', patience=2),  
    ModelCheckpoint('movie_weights.h5', save_best_only=True)]
history = model.fit([user_ids,movie_ids], ratings_list, epochs=30, validation_split=.1, verbose=2, callbacks=callbacks)

In [ ]:
min_val_loss, idx = min((val, idx) for (idx, val) in enumerate(history.history['val_loss']))
print 'Minimum RMSE at epoch', '{:d}'.format(idx+1), '=', '{:.4f}'.format(math.sqrt(min_val_loss))

In [7]:
# Use the pre-trained model
trained_model = clone_model(model)
# Load weights
trained_model.load_weights('weights.h5')

OSError: Unable to open file (unable to open file: name = 'weights.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
def predict_rating(user_id, movie_id):
    return trained_model.predict([np.array([user_id]), np.array([item_id])])[0][0]


In [ ]:
from random import choice
rand_user = choice(user_ids)

user_ratings = ratings[ratings['userId'] == rand_user][['userId', 'movieId', 'rating']]
user_ratings['prediction'] = user_ratings.apply(lambda x: predict_rating(rand_user, x['movie_id']), axis=1)
user_ratings.sort_values(by='rating', 
                         ascending=False).merge(movies, 
                                                on='movieId', 
                                                how='inner', 
                                                suffixes=['_u', '_m']).head(20)



In [ ]:
recommendations = ratings[ratings['movieId'].isin(user_ratings['movieId']) == False][['movieId']].drop_duplicates()
recommendations['prediction'] = recommendations.apply(lambda x: predict_rating(TEST_USER, x['movieId']), axis=1)
recommendations.sort_values(by='prediction',
                          ascending=False).merge(movies,
                                                 on='movieId',
                                                 how='inner',
                                                 suffixes=['_u', '_m']).head(20)